In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

## 훈련과 테스트 데이터
+ 머신러닝 모델을 만들기 위해서는 훈련(학습)에 필요한 데이터집합이 있어야 함
+ 그러나 전체 데이터 모두를 학습에 사용하면 과적합 문제 발생

### train/test 데이터 분할의 필요성 알아보기

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

#### 데이터 적재

In [6]:
iris = load_iris()
data = iris.data  # 특성/독립변수
label = iris.target  # 레이블/종속변수

#### 주어진 변수들로 의사결정나무 모델 생성을 위한 학습(fit)

In [7]:
dtclf = DecisionTreeClassifier()
dtclf.fit(data, label)

DecisionTreeClassifier()

#### 생성된 모델에 기존 변수를 이용해서 예측(predict)

In [8]:
pred = dtclf.predict(data)

#### 예측값과 실제를 비교하여 정확도 측정

In [9]:
accuracy_score(label, pred)

1.0

### 모델을 학습할 때 사용한 데이터를 모델을 평가할 때도 사용함 -> 정확도 : 1 -> 과적합

### 데이터를 학습용/평가용 데이터로 분할 1
+ 분할 비율 = 7 : 3
+ iris 데이터 건수 : 150 -> 105 : 45

In [11]:
train_data = iris.data[:105, ]
test_data = iris.data[105:, ]
train_target = iris.target[:105, ]
test_target = iris.target[105:, ]

In [12]:
dtclf = DecisionTreeClassifier()
dtclf.fit(train_data, train_target)

DecisionTreeClassifier()

In [13]:
pred = dtclf.predict(test_data)

In [14]:
accuracy_score(test_target, pred)

0.7333333333333333

### 데이터분할이 잘 되었는지 확인

In [17]:
pd.Series(train_target).value_counts()

0    50
1    50
2     5
dtype: int64

In [18]:
pd.Series(test_target).value_counts()

2    45
dtype: int64

In [19]:
train_target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

In [20]:
test_target

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2])

### 데이터를 앞에서부터 순서대로 나눴기 때문에 종속변수의 비율이 일정하지 않음

#### setosa, versicolor, verginica의 비율이 모두 같아야 하나 train에는 setosa, versicolor 위주로 test에는 verginica 위주로 데이터가 분할됨

### 데이터를 학습용/평가용 데이터로 분할 2
+ 독립변수의 속성들의 분포를 고려한 표본추출 필요
    - sklearn의 train_test_split 함수 사용
+ train_test_split(독립변수, 종속변수, 훈련데이터크기, 평가데이터크기, 계층추출여부(분류용), 시드값)

In [21]:
from sklearn.model_selection import train_test_split

In [30]:
Xtrain, Xtest, ytrain, ytest = train_test_split(iris.data, iris.target, train_size=0.7, test_size=0.3, random_state=2111040945)

In [28]:
pd.Series(ytrain).value_counts()

2    35
1    35
0    35
dtype: int64

In [31]:
pd.Series(ytest).value_counts()

2    19
1    15
0    11
dtype: int64

In [34]:
dtclf2 = DecisionTreeClassifier()
dtclf2.fit(Xtrain, ytrain)

DecisionTreeClassifier()

In [35]:
pred2 = dtclf2.predict(Xtest)

In [36]:
accuracy_score(ytest, pred2)

0.9333333333333333

#### 계층추출 - 과적합 완화

In [37]:
Xtrain, Xtest, ytrain, ytest = train_test_split(iris.data, iris.target, train_size=0.7, test_size=0.3, stratify=iris.target, random_state=2111040945)

In [38]:
dtclf3 = DecisionTreeClassifier()
dtclf3.fit(Xtrain, ytrain)

DecisionTreeClassifier()

In [39]:
pred3 = dtclf3.predict(Xtest)

In [40]:
accuracy_score(ytest, pred3)

0.9111111111111111

### 데이터를 학습용/평가용 데이터로 분할 3
+ 데이터수가 작은 경우 데이터의 일부인 평가 데이터도 작음
    - 성능평가의 신뢰도 의심
+ 데이터를 동일한 크기의 k개로 나누고 훈련/평가데이터로 구분지어 순환적인 훈련 및 평가를 k번 실시
    - K fold 교차검증
+ cross_val_score(모델객체, 독립변수, 종속변수, 평가방법, k값)

In [41]:
from sklearn.model_selection import cross_val_score

In [43]:
dtclf4 = DecisionTreeClassifier()
scores = cross_val_score(dtclf4, iris.data, iris.target, scoring='accuracy', cv=5)

In [44]:
scores

array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ])

In [46]:
np.mean(scores)

0.9600000000000002